# Analysis of Facebook Large Page-Page Dataset using Mutual Consumer Behavioral page liking & Community Detection based upon Graph/SubGraph Structure 


**Team details**
- Kruthika Suresh &emsp;&emsp;&emsp; PES1UG19CS233
- Yousha Mahamuni &emsp;&ensp;&nbsp; PES2UG19CS468
- Shrikar Madhu &emsp;&emsp;&emsp;&ensp; PES1UG19CS470
- Smriti Tilak &emsp;&emsp;&emsp;&emsp;&emsp; PES1UG19CS486

In [ ]:
!pip install node2vec

In [ ]:
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sp
import collections
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, confusion_matrix, classification_report
from node2vec import Node2Vec as n2v

#Dataset Description

> This webgraph is a page-page graph of verified Facebook sites. Nodes represent official Facebook pages while the links are mutual likes between sites. Node features are extracted from the site descriptions that the page owners created to summarize the purpose of the site. This graph was collected through the Facebook Graph API in November 2017 and restricted to pages from 4 categories which are defined by Facebook. These categories are: politicians, governmental organizations, television shows and companies. 

> The links are formed based on mutual likes between pages i.e., an edge is present between nodes if a user has liked both nodes

The dataset has been taken from SNAP  
> Link: [Facebook Large Page-Page Network](https://snap.stanford.edu/data/facebook-large-page-page-network.html)

Gephi Visualisations can be viewed in the slides: [Link](https://docs.google.com/presentation/d/1kGk5RWkH5xxmSVGHWpztKQgYqHe9uSs4/edit?usp=sharing&ouid=115873317750525938978&rtpof=true&sd=true)

In [ ]:
edges = pd.read_csv("musae_facebook_edges.csv")
target = pd.read_csv("musae_facebook_target.csv")

In [ ]:
edges.head()

In [ ]:
target.head()

In [ ]:
target['page_type'].hist(color = "purple")

Not all classes have the same number of the nodes. TV Show has the least nodes and Government pages are the highest in the dataset

In [ ]:
print(target.groupby(['page_type']).count())

#Creating the Graph

In [ ]:
G = nx.Graph()

#Adding nodes with attribute = page_type
for i, j in zip(target['id'], target['page_type']):
  G.add_node(i, page_type = j)

#Adding edges between the nodes
for i, j in zip(edges['id_1'], edges['id_2']):
  G.add_edge(i, j)

## Graph Statistics and Visualisation

In [ ]:
print("Number of nodes in the graph: ", G.number_of_nodes())
print("Number of edges in the graph: ", G.number_of_edges())

In [ ]:
nx.is_directed(G)

The graph we have is undirected

In [ ]:
print("Number of self loops in the graph: ", nx.number_of_selfloops(G))
print("Number of nodes with self loops: ", len(list(nx.nodes_with_selfloops(G))))
print("Density of the graph: ", nx.density(G))
print("Transitivity of the graph: ", nx.transitivity(G))

A page does not have more than one link to itself as the number of self loops is equal to the number of nodes with self loops. The graph is highly sparse.

In [ ]:
nx.attribute_assortativity_coefficient(G, "page_type")

The Assortativity is very high which could mean people who liked pages of a certain type mostly tend to like similar pages

In [ ]:
'''plt.figure(figsize=(100, 100))
nx.draw(G, with_labels = True)'''

Since the above command takes approx 30 mins to execute, we have linked the output below.

[Link to the network graph](https://drive.google.com/file/d/15XdBDt-ZQodME5D4DbXS2qaaNkP8akGy/view?usp=sharing)

In [ ]:
degree_sequence = sorted([deg for node, deg in G.degree()], reverse=True)
degree_count = collections.Counter(degree_sequence)
deg, count = zip(*degree_count.items())

fig, ax = plt.subplots(figsize=(20, 10))
plt.bar(deg, count, width=0.80, color="purple")
plt.title("Degree Distribution for the Graph")
plt.ylabel("Count")
plt.xlabel("Degree")
plt.xlim(0, 200)
plt.show()

The degree distribution follows power-law, hence, it is scale-free.

In [ ]:
d = dict(G.degree())
isolates = [k for k,v in d.items() if float(v) == 0]
print("Number of nodes with degree zero: ", len(isolates))

##Sampling the Graph

> The graph has over 20000 nodes and over 100000 edges and performing analysis on it is computationally expensive. This is why we chose to subsample the graph. We have decided to drop nodes that have a degree < 40. It effectively brought down the node count to 1977 which was manageable.

In [ ]:
#Get all nodes with degree < 40

deg = [k for k,v in d.items() if float(v) < 40]
len(deg)

In [ ]:
print("Before removing nodes: ")
print("Number of nodes in the graph: ", G.number_of_nodes())
print("Number of edges in the graph: ", G.number_of_edges())

G.remove_nodes_from(deg)
print()
print("After removing nodes: ")
print("Number of nodes in the graph: ", G.number_of_nodes())
print("Number of edges in the graph: ", G.number_of_edges())

We have removed the nodes with a degree less than 40 in order to reduce graph size

### Sampled Graph Statistics

In [ ]:
print("Density of the graph: ", nx.density(G))
print("Assortativity: ", nx.attribute_assortativity_coefficient(G, "page_type"))
print("Transitivity of the graph: ", nx.transitivity(G))
print("Number of self loops in the graph: ", nx.number_of_selfloops(G))
print("Number of nodes with self loops: ", len(list(nx.nodes_with_selfloops(G))))

The new graph has a higher density which helps with the computation. Assortativity approximately remains the same

In [ ]:
degree_sequence = sorted([deg for node, deg in G.degree()], reverse=True)
degree_count = collections.Counter(degree_sequence)
deg, count = zip(*degree_count.items())

fig, ax = plt.subplots(figsize=(20, 10))
plt.bar(deg, count, width=0.80, color="purple")
plt.title("Degree Distribution for the Sampled Graph")
plt.ylabel("Count")
plt.xlabel("Degree")
plt.xlim(0, 200)
plt.show()

The distribution is still skewed but not as heavily skewed as in the case of the power law distribution observed earlier

### Plotting the Subgraphs for the 4 classes

In [ ]:
attr = dict(nx.get_node_attributes(G, "page_type"))

govt_nodes = [k for k, v in attr.items() if v == "government"]
tv_nodes = [k for k, v in attr.items() if v == "tvshow"]
company_nodes = [k for k, v in attr.items() if v == "company"]
politician_nodes = [k for k, v in attr.items() if v == "politician"]

#### Government Subgraph

In [ ]:
G_govt = G.subgraph(govt_nodes)
plt.figure(figsize=(100, 100))
nx.draw(G_govt, with_labels = True)

In [ ]:
nx.info(G_govt)

The government class retained the most number of nodes as compared to the other sub-groups. This would mean that the government pages are the most mutually liked. We can also observe that most nodes are clustered together this could be due to the fact that people interested in a particular diplomatic opinion tend to follow/like similar pages with support their opinions only.

#### TV Shows Subgraph

In [ ]:
G_tv = G.subgraph(tv_nodes)
plt.figure(figsize=(100, 100))
nx.draw(G_tv, with_labels = True)

In [ ]:
nx.info(G_tv)

The TV Shows class only retained 117 nodes after subsampling. A few dense clusters can be observed in the graph. This could be because people tend to follow shows of a similar genre or the popular shows could be clustered together

#### Company Subgraph

In [ ]:
G_company = G.subgraph(company_nodes)
plt.figure(figsize=(100, 100))
nx.draw(G_company, with_labels = True)

The company subgraph

In [ ]:
nx.info(G_company)

#### Politician Subgraph

In [ ]:
G_politician = G.subgraph(politician_nodes)
plt.figure(figsize=(100, 100))
nx.draw(G_politician, with_labels = True)

In [ ]:
nx.info(G_politician)

# Community Detection in Subgraphs

In [ ]:
!pip uninstall community
!pip install python-louvain

import matplotlib.colors as mpcol
import matplotlib.cm as cm

In [ ]:
import time
from community import community_louvain

def find_communities(G):
    start_time = time.time()
    partition = community_louvain.best_partition(G)
    part_dict = {}
    values = []
    for node in G.nodes():
        values.append(partition.get(node))
        part_dict.update({node:partition.get(node)})
    communities_louvain= max(values)+1
    end_time = time.time()
    mod_louvain = community_louvain.modularity(partition, G)
    print('Communities found using the Louvain algorithm: {} \nModularity: {} \nTime for finding the communities: {} s'.format(communities_louvain, mod_louvain,round((end_time-start_time),3)))
    return part_dict

## Communities in the Government Subgraph

In [ ]:
comm_govt = find_communities(G_govt)

In [ ]:
#Set the node attributes ussing the partition dictionary.
nx.set_node_attributes(G_govt, comm_govt, 'community')

# Get a list of len(nodes) with their corresponding community
communities = [(G_govt.nodes()[i]['community']) for i in G_govt.nodes()]

In [ ]:
colours = ['#E1CF3F','#F47757','#FD4581','#97577C','#BDA7A9','#E1CF3F','#F47757','#FD4581',
                 '#e44623','#e45a6a','#c9d3e6','#7d513d',
                 '#e65949','#d6b240','#382a29','#d8d4c9',
                 '#e4cc34','#ccb42c','#bc8ca4','#3c84c4',
                 '#dd4d3d','#52172f','#63494a','#e2d5d3',
                 '#f7abcc','#e085a1','#943d39','#2d1d19']

Barragan = mpcol.ListedColormap(colours, name='Barragan')

In [ ]:
com = [x[1] for x in G_govt.nodes(data='community')]
norm = mpcol.Normalize(vmin=min(com), vmax=max(com), clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=Barragan)
nc=[mapper.to_rgba(x) for x in com]

In [ ]:
plt.figure(figsize=(100, 100))
nx.draw(G_govt, node_color = nc, with_labels = False)

In [ ]:
govt_commlist = dict(nx.get_node_attributes(G_govt, "community"))
govt_commlist1 = {}
for k, v in govt_commlist.items():
  if v not in govt_commlist1:
    govt_commlist1[v] = [k]
  else:
    govt_commlist1[v].append(k)
print(govt_commlist1)

In [ ]:
final_govt = {}
for k in govt_commlist1:
  final_govt[k] = []
  for v in govt_commlist1[k]:
    r = target['page_name'][v]
    final_govt[k].append(r)
print(final_govt)

## Communities in the TV Shows Subgraph

In [ ]:
comm_tv = find_communities(G_tv)

In [ ]:
#Set the node attributes ussing the partition dictionary.
nx.set_node_attributes(G_tv, comm_tv, 'community')

# Get a list of len(nodes) with their corresponding community
communities = [(G_tv.nodes()[i]['community']) for i in G_tv.nodes()]

In [ ]:
com = [x[1] for x in G_tv.nodes(data='community')]
norm = mpcol.Normalize(vmin=min(com), vmax=max(com), clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=Barragan)
nc=[mapper.to_rgba(x) for x in com]

In [ ]:
plt.figure(figsize=(100, 100))
nx.draw(G_tv, node_color = nc, with_labels = False)

In [ ]:
tv_commlist = dict(nx.get_node_attributes(G_tv, "community"))
tv_commlist1 = {}
for k, v in tv_commlist.items():
  if v not in tv_commlist1:
    tv_commlist1[v] = [k]
  else:
    tv_commlist1[v].append(k)
print(tv_commlist1)

In [ ]:
final_tv = {}
for k in tv_commlist1:
  final_tv[k] = []
  for v in tv_commlist1[k]:
    r = target['page_name'][v]
    final_tv[k].append(r)
print(final_tv)

## Communities in the Company Subgraph

In [ ]:
comm_company = find_communities(G_company)

In [ ]:
#Set the node attributes ussing the partition dictionary.
nx.set_node_attributes(G_company, comm_company, 'community')

# Get a list of len(nodes) with their corresponding community
communities = [(G_company.nodes()[i]['community']) for i in G_company.nodes()]

In [ ]:
com = [x[1] for x in G_company.nodes(data='community')]
norm = mpcol.Normalize(vmin=min(com), vmax=max(com), clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=Barragan)
nc=[mapper.to_rgba(x) for x in com]

In [ ]:
plt.figure(figsize=(100, 100))
nx.draw(G_company, node_color = nc, with_labels = False)

In [ ]:
comp_commlist = dict(nx.get_node_attributes(G_company, "community"))
comp_commlist1 = {}
for k, v in comp_commlist.items():
  if v not in comp_commlist1:
    comp_commlist1[v] = [k]
  else:
    comp_commlist1[v].append(k)
print(comp_commlist1)

In [ ]:
final_comp = {}
for k in comp_commlist1:
  final_comp[k] = []
  for v in comp_commlist1[k]:
    r = target['page_name'][v]
    final_comp[k].append(r)
print(final_comp)

## Communities in the Politician Subgraph

In [ ]:
comm_politician = find_communities(G_politician)

In [ ]:
#Set the node attributes ussing the partition dictionary.
nx.set_node_attributes(G_politician, comm_politician, 'community')

# Get a list of len(nodes) with their corresponding community
communities = [(G_politician.nodes()[i]['community']) for i in G_politician.nodes()]

In [ ]:
com = [x[1] for x in G_politician.nodes(data='community')]
norm = mpcol.Normalize(vmin=min(com), vmax=max(com), clip=True)
mapper = cm.ScalarMappable(norm=norm, cmap=Barragan)
nc=[mapper.to_rgba(x) for x in com]

In [ ]:
plt.figure(figsize=(100, 100))
nx.draw(G_politician, node_color = nc, with_labels = False)

In [ ]:
pol_commlist = dict(nx.get_node_attributes(G_politician, "community"))
pol_commlist1 = {}
for k, v in pol_commlist.items():
  if v not in pol_commlist1:
    pol_commlist1[v] = [k]
  else:
    pol_commlist1[v].append(k)
print(pol_commlist1)

In [ ]:
final_pol = {}
for k in pol_commlist1:
  final_pol[k] = []
  for v in pol_commlist1[k]:
    r = target['page_name'][v]
    final_pol[k].append(r)
print(final_pol)

# Node level Classification using Node2Vec


In [ ]:
g_emb = n2v(G, dimensions=16)


Computing transition probabilities:   0%|          | 0/1977 [00:00<?, ?it/s]

Generating walks (CPU: 1):  90%|█████████ | 9/10 [01:08<00:08,  8.11s/it]

In [ ]:
WINDOW = 1 # Node2Vec fit window
MIN_COUNT = 1 # Node2Vec min. count
BATCH_WORDS = 4 # Node2Vec batch words

mdl = g_emb.fit(
    window=WINDOW,
    min_count=MIN_COUNT,
    batch_words=BATCH_WORDS
)

emb_df = (
    pd.DataFrame(
        [mdl.wv.get_vector(str(n)) for n in G.nodes()],
        index = G.nodes
    )
)





In [ ]:
emb_df = emb_df.merge(
    target[['id', 'page_name','page_type']].set_index('id'),
    left_index = True,
    right_index = True
)
emb_df.head()


In [ ]:
print(emb_df.page_type.value_counts().head())


In [ ]:
ft_cols = emb_df.drop(columns = ['page_type','page_name']).columns.tolist()
target_col = 'page_type'

# train test split
x = emb_df[ft_cols].values
y = emb_df[target_col].values

x_train, x_test, y_train, y_test = train_test_split(
    x, 
    y,
    test_size = 0.3
)

# GBC classifier
clf = GradientBoostingClassifier()

# train the model
clf.fit(x_train, y_train)

In [ ]:
def clf_eval(clf, x_test, y_test):
    '''
    This function will evaluate a sk-learn multi-class classification model based on its
    x_test and y_test values
    
    params:
        clf (Model) : The model you wish to evaluate the performance of
        x_test (Array) : Result of the train test split
        y_test (Array) : Result of the train test split
    
    returns:
        This function will return the following evaluation metrics:
            - Accuracy Score
            - Matthews Correlation Coefficient
            - Classification Report
            - Confusion Matrix
    
    example:
        clf_eval(
            clf,
            x_test,
            y_test
        )
    '''
    y_pred = clf.predict(x_test)
    y_true = y_test
    
    y_pred = clf.predict(x_test)
    test_acc = accuracy_score(y_test, y_pred)
    print("Testing Accuracy : ", test_acc)
    
    print("MCC Score : ", matthews_corrcoef(y_true, y_pred))
    
    print("Classification Report : ")
    print(classification_report(y_test, clf.predict(x_test)))
    
    print(confusion_matrix(y_pred,y_test))
    
clf_eval(
    clf,
    x_test,
    y_test
)

In [ ]:
G.nodes()

In [ ]:
pred_ft = [mdl.wv.get_vector(str('14'))]
clf.predict(pred_ft)[0]

#GraphML for Link Prediction

## Label Encoding

In [ ]:
attr = dict(nx.get_node_attributes(G, "page_type"))
attr2={'government':0,'politician':1,'tvshow':2,'company':3}


In [ ]:
for x, y in attr.items():
    attr[x] = attr2[y]

In [ ]:
list1=list(attr.values())


In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-geometric
!pip install class-resolver

In [ ]:
from torch_geometric.utils.convert import to_networkx, from_networkx
pyg = from_networkx(G_company)
pyg.edge_index

In [ ]:
attr=dict(nx.get_node_attributes(G_company,"page_type"))
attr2={'government':0,'politician':1,'tvshow':2,'company':3}
for x,y in attr.items():
  attr[x]=attr2[y]
list3=list(attr.values())
print(list3)

## Data preprocessing

In [ ]:
from typing import Callable, Optional

import torch

from torch_geometric.data import Data, InMemoryDataset


class MyDataset(InMemoryDataset):
    
    def __init__(self, transform: Optional[Callable] = None):
        super(MyDataset, self).__init__('.', transform)

        edge_index = pyg.edge_index
        y = torch.tensor(list3)

        x = torch.eye(y.size(0), dtype=torch.float)

        # Select a single training node for each community
        # (we just use the first one).
        train_mask = torch.zeros(y.size(0), dtype=torch.bool)
        #for i in range(int(y.max()) + 1):
        train_mask[(y == 3).nonzero(as_tuple=False)[0]] = True

        data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask)

        self.data, self.slices = self.collate([data])

In [ ]:
dataset=MyDataset()

In [ ]:
dataset.data

## GCN Model

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score,roc_auc_score
from sklearn.preprocessing import normalize
import torch
import torch.nn.functional as F
from scipy.sparse.csgraph import shortest_path
from torch.nn import BCEWithLogitsLoss, Conv1d, MaxPool1d, ModuleList
from itertools import chain
import math

from torch_geometric.data import InMemoryDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, GCNConv, global_sort_pool
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import k_hop_subgraph, to_scipy_sparse_matrix

class SEALDataset(InMemoryDataset):
    def __init__(self, dataset, num_hops, split='train'):
        self.data = dataset[0]
        self.num_hops = num_hops
        super().__init__('.')
        index = ['train', 'val', 'test'].index(split)
        self.data, self.slices = torch.load(self.processed_paths[index])

    @property
    def processed_file_names(self):
        return ['SEAL_train_data.pt', 'SEAL_val_data.pt', 'SEAL_test_data.pt']

    def process(self):
        transform = RandomLinkSplit(num_val=0.05, num_test=0.1,
                                    is_undirected=True, split_labels=True)
        train_data, val_data, test_data = transform(self.data)

        self._max_z = 0

        # Collect a list of subgraphs for training, validation and testing:
        train_pos_data_list = self.extract_enclosing_subgraphs(
            train_data.edge_index, train_data.pos_edge_label_index, 1)
        train_neg_data_list = self.extract_enclosing_subgraphs(
            train_data.edge_index, train_data.neg_edge_label_index, 0)

        val_pos_data_list = self.extract_enclosing_subgraphs(
            val_data.edge_index, val_data.pos_edge_label_index, 1)
        val_neg_data_list = self.extract_enclosing_subgraphs(
            val_data.edge_index, val_data.neg_edge_label_index, 0)

        test_pos_data_list = self.extract_enclosing_subgraphs(
            test_data.edge_index, test_data.pos_edge_label_index, 1)
        test_neg_data_list = self.extract_enclosing_subgraphs(
            test_data.edge_index, test_data.neg_edge_label_index, 0)

        # Convert node labeling to one-hot features.
        for data in chain(train_pos_data_list, train_neg_data_list,
                          val_pos_data_list, val_neg_data_list,
                          test_pos_data_list, test_neg_data_list):
            # We solely learn links from structure, dropping any node features:
            data.x = F.one_hot(data.z, self._max_z + 1).to(torch.float)

        torch.save(self.collate(train_pos_data_list + train_neg_data_list),
                   self.processed_paths[0])
        torch.save(self.collate(val_pos_data_list + val_neg_data_list),
                   self.processed_paths[1])
        torch.save(self.collate(test_pos_data_list + test_neg_data_list),
                   self.processed_paths[2])

    def extract_enclosing_subgraphs(self, edge_index, edge_label_index, y):
        data_list = []
        for src, dst in edge_label_index.t().tolist():
            sub_nodes, sub_edge_index, mapping, _ = k_hop_subgraph(
                [src, dst], self.num_hops, edge_index, relabel_nodes=True)
            src, dst = mapping.tolist()

            # Remove target link from the subgraph.
            mask1 = (sub_edge_index[0] != src) | (sub_edge_index[1] != dst)
            mask2 = (sub_edge_index[0] != dst) | (sub_edge_index[1] != src)
            sub_edge_index = sub_edge_index[:, mask1 & mask2]

            # Calculate node labeling.
            z = self.drnl_node_labeling(sub_edge_index, src, dst,
                                        num_nodes=sub_nodes.size(0))
            #print(sub_nodes)
            data = Data(x=self.data.x[sub_nodes], z=z,
                        edge_index=sub_edge_index, y=y)
            data_list.append(data)

        return data_list

    def drnl_node_labeling(self, edge_index, src, dst, num_nodes=None):
        # Double-radius node labeling (DRNL).
        src, dst = (dst, src) if src > dst else (src, dst)
        adj = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes).tocsr()

        idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
        adj_wo_src = adj[idx, :][:, idx]

        idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
        adj_wo_dst = adj[idx, :][:, idx]

        dist2src = shortest_path(adj_wo_dst, directed=False, unweighted=True,
                                 indices=src)
        dist2src = np.insert(dist2src, dst, 0, axis=0)
        dist2src = torch.from_numpy(dist2src)

        dist2dst = shortest_path(adj_wo_src, directed=False, unweighted=True,
                                 indices=dst - 1)
        dist2dst = np.insert(dist2dst, src, 0, axis=0)
        dist2dst = torch.from_numpy(dist2dst)

        dist = dist2src + dist2dst
        dist_over_2, dist_mod_2 = dist // 2, dist % 2

        z = 1 + torch.min(dist2src, dist2dst)
        z += dist_over_2 * (dist_over_2 + dist_mod_2 - 1)
        z[src] = 1.
        z[dst] = 1.
        z[torch.isnan(z)] = 0.

        self._max_z = max(int(z.max()), self._max_z)

        return z.to(torch.long)
train_dataset = SEALDataset(dataset, num_hops=2, split='train')
val_dataset = SEALDataset(dataset, num_hops=2, split='val')
test_dataset = SEALDataset(dataset, num_hops=2, split='test')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

class DGCNN(torch.nn.Module):
    def __init__(self, hidden_channels, num_layers, GNN=GCNConv, k=0.6):
        super().__init__()

        if k < 1:  # Transform percentile to number.
            num_nodes = sorted([data.num_nodes for data in train_dataset])
            k = num_nodes[int(math.ceil(k * len(num_nodes))) - 1]
            k = max(10, k)
        self.k = int(k)

        self.convs = ModuleList()
        self.convs.append(GNN(train_dataset.num_features, hidden_channels))
        for i in range(0, num_layers - 1):
            self.convs.append(GNN(hidden_channels, hidden_channels))
        self.convs.append(GNN(hidden_channels, 1))

        conv1d_channels = [16, 32]
        total_latent_dim = hidden_channels * num_layers + 1
        conv1d_kws = [total_latent_dim, 5]
        self.conv1 = Conv1d(1, conv1d_channels[0], conv1d_kws[0],
                            conv1d_kws[0])
        self.maxpool1d = MaxPool1d(2, 2)
        self.conv2 = Conv1d(conv1d_channels[0], conv1d_channels[1],
                            conv1d_kws[1], 1)
        dense_dim = int((self.k - 2) / 2 + 1)
        dense_dim = (dense_dim - conv1d_kws[1] + 1) * conv1d_channels[1]
        self.mlp = MLP([dense_dim, 128, 1], dropout=0.5, batch_norm=False)

    def forward(self, x, edge_index, batch):
        xs = [x]
        for conv in self.convs:
            xs += [conv(xs[-1], edge_index).tanh()]
        x = torch.cat(xs[1:], dim=-1)

        # Global pooling.
        x = global_sort_pool(x, batch, self.k)
        x = x.unsqueeze(1)  # [num_graphs, 1, k * hidden]
        x = self.conv1(x).relu()
        x = self.maxpool1d(x)
        x = self.conv2(x).relu()
        x = x.view(x.size(0), -1)  # [num_graphs, dense_dim]

        return self.mlp(x)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCNN(hidden_channels=32, num_layers=3).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = BCEWithLogitsLoss()


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out.view(-1), data.y.to(torch.float))
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs

    return total_loss / len(train_dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    y_pred, y_true = [], []
    for data in loader:
        data = data.to(device)
        logits = model(data.x, data.edge_index, data.batch)
        y_pred.append(logits.view(-1).cpu())
        y_true.append(data.y.view(-1).cpu().to(torch.float))

    return roc_auc_score(torch.cat(y_true), torch.cat(y_pred))


best_val_auc = test_aucGCN = 0
for epoch in range(1, 51):
    lossGCN = train()
    val_auc = test(val_loader)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        test_aucGCN = test(test_loader)
    print(f'Epoch: {epoch:02d}, Loss: {lossGCN:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_aucGCN:.4f}')


## GAT Model

In [ ]:
from torch_geometric.nn.conv.gat_conv import GATConv
import torch
import torch.nn.functional as F
from scipy.sparse.csgraph import shortest_path
from torch.nn import BCEWithLogitsLoss, Conv1d, MaxPool1d, ModuleList
from itertools import chain
import math

from torch_geometric.data import InMemoryDataset
from torch_geometric.datasets import Planetoid
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, GCNConv, global_sort_pool,SAGEConv,GATConv
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import k_hop_subgraph, to_scipy_sparse_matrix

class SEALDataset(InMemoryDataset):
    def __init__(self, dataset, num_hops, split='train'):
        self.data = dataset[0]
        self.num_hops = num_hops
        super().__init__('.')
        index = ['train', 'val', 'test'].index(split)
        self.data, self.slices = torch.load(self.processed_paths[index])

    @property
    def processed_file_names(self):
        return ['SEAL_train_data.pt', 'SEAL_val_data.pt', 'SEAL_test_data.pt']

    def process(self):
        transform = RandomLinkSplit(num_val=0.05, num_test=0.1,
                                    is_undirected=True, split_labels=True)
        train_data, val_data, test_data = transform(self.data)

        self._max_z = 0

        # Collect a list of subgraphs for training, validation and testing:
        train_pos_data_list = self.extract_enclosing_subgraphs(
            train_data.edge_index, train_data.pos_edge_label_index, 1)
        train_neg_data_list = self.extract_enclosing_subgraphs(
            train_data.edge_index, train_data.neg_edge_label_index, 0)

        val_pos_data_list = self.extract_enclosing_subgraphs(
            val_data.edge_index, val_data.pos_edge_label_index, 1)
        val_neg_data_list = self.extract_enclosing_subgraphs(
            val_data.edge_index, val_data.neg_edge_label_index, 0)

        test_pos_data_list = self.extract_enclosing_subgraphs(
            test_data.edge_index, test_data.pos_edge_label_index, 1)
        test_neg_data_list = self.extract_enclosing_subgraphs(
            test_data.edge_index, test_data.neg_edge_label_index, 0)

        # Convert node labeling to one-hot features.
        for data in chain(train_pos_data_list, train_neg_data_list,
                          val_pos_data_list, val_neg_data_list,
                          test_pos_data_list, test_neg_data_list):
            # We solely learn links from structure, dropping any node features:
            data.x = F.one_hot(data.z, self._max_z + 1).to(torch.float)

        torch.save(self.collate(train_pos_data_list + train_neg_data_list),
                   self.processed_paths[0])
        torch.save(self.collate(val_pos_data_list + val_neg_data_list),
                   self.processed_paths[1])
        torch.save(self.collate(test_pos_data_list + test_neg_data_list),
                   self.processed_paths[2])

    def extract_enclosing_subgraphs(self, edge_index, edge_label_index, y):
        data_list = []
        for src, dst in edge_label_index.t().tolist():
            sub_nodes, sub_edge_index, mapping, _ = k_hop_subgraph(
                [src, dst], self.num_hops, edge_index, relabel_nodes=True)
            src, dst = mapping.tolist()

            # Remove target link from the subgraph.
            mask1 = (sub_edge_index[0] != src) | (sub_edge_index[1] != dst)
            mask2 = (sub_edge_index[0] != dst) | (sub_edge_index[1] != src)
            sub_edge_index = sub_edge_index[:, mask1 & mask2]

            # Calculate node labeling.
            z = self.drnl_node_labeling(sub_edge_index, src, dst,
                                        num_nodes=sub_nodes.size(0))
            #print(sub_nodes)
            data = Data(x=self.data.x[sub_nodes], z=z,
                        edge_index=sub_edge_index, y=y)
            data_list.append(data)

        return data_list

    def drnl_node_labeling(self, edge_index, src, dst, num_nodes=None):
        # Double-radius node labeling (DRNL).
        src, dst = (dst, src) if src > dst else (src, dst)
        adj = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes).tocsr()

        idx = list(range(src)) + list(range(src + 1, adj.shape[0]))
        adj_wo_src = adj[idx, :][:, idx]

        idx = list(range(dst)) + list(range(dst + 1, adj.shape[0]))
        adj_wo_dst = adj[idx, :][:, idx]

        dist2src = shortest_path(adj_wo_dst, directed=False, unweighted=True,
                                 indices=src)
        dist2src = np.insert(dist2src, dst, 0, axis=0)
        dist2src = torch.from_numpy(dist2src)

        dist2dst = shortest_path(adj_wo_src, directed=False, unweighted=True,
                                 indices=dst - 1)
        dist2dst = np.insert(dist2dst, src, 0, axis=0)
        dist2dst = torch.from_numpy(dist2dst)

        dist = dist2src + dist2dst
        dist_over_2, dist_mod_2 = dist // 2, dist % 2

        z = 1 + torch.min(dist2src, dist2dst)
        z += dist_over_2 * (dist_over_2 + dist_mod_2 - 1)
        z[src] = 1.
        z[dst] = 1.
        z[torch.isnan(z)] = 0.

        self._max_z = max(int(z.max()), self._max_z)

        return z.to(torch.long)
train_dataset = SEALDataset(dataset, num_hops=2, split='train')
val_dataset = SEALDataset(dataset, num_hops=2, split='val')
test_dataset = SEALDataset(dataset, num_hops=2, split='test')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

class DGCNN(torch.nn.Module):
    def __init__(self, hidden_channels, num_layers, GNN=GATConv, k=0.6):
        super().__init__()

        if k < 1:  # Transform percentile to number.
            num_nodes = sorted([data.num_nodes for data in train_dataset])
            k = num_nodes[int(math.ceil(k * len(num_nodes))) - 1]
            k = max(10, k)
        self.k = int(k)

        self.convs = ModuleList()
        self.convs.append(GNN(train_dataset.num_features, hidden_channels))
        for i in range(0, num_layers - 1):
            self.convs.append(GNN(hidden_channels, hidden_channels))
        self.convs.append(GNN(hidden_channels, 1))

        conv1d_channels = [16, 32]
        total_latent_dim = hidden_channels * num_layers + 1
        conv1d_kws = [total_latent_dim, 5]
        self.conv1 = Conv1d(1, conv1d_channels[0], conv1d_kws[0],
                            conv1d_kws[0])
        self.maxpool1d = MaxPool1d(2, 2)
        self.conv2 = Conv1d(conv1d_channels[0], conv1d_channels[1],
                            conv1d_kws[1], 1)
        dense_dim = int((self.k - 2) / 2 + 1)
        dense_dim = (dense_dim - conv1d_kws[1] + 1) * conv1d_channels[1]
        self.mlp = MLP([dense_dim, 128, 1], dropout=0.5, batch_norm=False)

    def forward(self, x, edge_index, batch):
        xs = [x]
        for conv in self.convs:
            xs += [conv(xs[-1], edge_index).tanh()]
        x = torch.cat(xs[1:], dim=-1)

        # Global pooling.
        x = global_sort_pool(x, batch, self.k)
        x = x.unsqueeze(1)  # [num_graphs, 1, k * hidden]
        x = self.conv1(x).relu()
        x = self.maxpool1d(x)
        x = self.conv2(x).relu()
        x = x.view(x.size(0), -1)  # [num_graphs, dense_dim]

        return self.mlp(x)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DGCNN(hidden_channels=32, num_layers=3).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0001)
criterion = BCEWithLogitsLoss()


def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out.view(-1), data.y.to(torch.float))
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs

    return total_loss / len(train_dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    y_pred, y_true = [], []
    for data in loader:
        data = data.to(device)
        logits = model(data.x, data.edge_index, data.batch)
        y_pred.append(logits.view(-1).cpu())
        y_true.append(data.y.view(-1).cpu().to(torch.float))

    return roc_auc_score(torch.cat(y_true), torch.cat(y_pred))


best_val_auc = test_aucGAT = 0
for epoch in range(1, 51):
    lossGAT = train()
    val_auc = test(val_loader)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        test_aucGAT = test(test_loader)
    print(f'Epoch: {epoch:02d}, Loss: {lossGAT:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_aucGAT:.4f}')

In [ ]:
print("AUC_GCN of Company Sub Graph")
print(test_aucGCN)
print("Loss_GCN of Company Sub Graph ")
print(lossGCN)

print("AUC_GAT of Company Sub Graph")
print(test_aucGAT)
print("Loss_GAT of Company Sub Graph ")
print(lossGAT)

AUC Values for the GAT and GCN model are comparable

#Centrality Measures


In [ ]:
print("Density of the Government Sub Graph")
print(nx.density(G_govt))
print("Density of the Politician Sub Graph")
print(nx.density(G_politician))
print("Density of the Tv Shows Sub Graph")
print(nx.density(G_tv))
print("Density of the Company Sub Graph")
print(nx.density(G_company))


The TV shows subgraph is the most dense and the government subgraph (which has the highest nodes) has the least density. But if we observe, company subgraph has the most nodes and yet it has the second lowest density which would imply, node count has minimum impact on density.

In [ ]:
print("Average Clustering of the Government Sub Graph")
print(nx.average_clustering(G_govt))
print("Average Clustering of the Politician Sub Graph")
print(nx.average_clustering(G_politician))
print("Average Clustering of the Tv Shows Sub Graph")
print(nx.average_clustering(G_tv))
print("Average Clustering of the Company Sub Graph")
print(nx.average_clustering(G_company))

Average clustering coefficient is highest for TV shows graph. This says a lot about the domain itself. This implies that majority of people follow mutual shows. 

In [ ]:
print("Closeness centrality of the Government Sub Graph")
closeness1 = nx.closeness_centrality(G_govt)
print(max(sorted(closeness1.items(), key=lambda x: x[1], reverse=True)[0:]))
print("Closeness Centrality of the Politician Sub Graph")
closeness2 = nx.closeness_centrality(G_politician)
print(max(sorted(closeness2.items(), key=lambda x: x[1], reverse=True)[0:]))
print("Closeness Centrality of the Tv Shows Sub Graph")
closeness3 = nx.closeness_centrality(G_tv)
print(max(sorted(closeness3.items(), key=lambda x: x[1], reverse=True)[0:]))
print("Closeness Centrality of the Company Sub Graph")
closeness4 = nx.closeness_centrality(G_company)
print(max(sorted(closeness4.items(), key=lambda x: x[1], reverse=True)[0:]))

In [ ]:
print("Page Rank of the Government Sub Graph")
pr1 = nx.pagerank(G_govt, alpha=0.85)
print(max(sorted(pr1.items(), key=lambda x: x[1], reverse=True)[0:]))
print(target)
print("Page Rank of the Politician Sub Graph")
pr2 = nx.pagerank(G_politician, alpha=0.85)
print(max(sorted(pr2.items(), key=lambda x: x[1], reverse=True)[0:]))
print("Page Rank of the Tv Shows Sub Graph")
pr3 = nx.pagerank(G_tv, alpha=0.85)
print(max(sorted(pr3.items(), key=lambda x: x[1], reverse=True)[0:]))
print("Page Rank of the Company Sub Graph")
pr4 = nx.pagerank(G_company, alpha=0.85)
print(max(sorted(pr4.items(), key=lambda x: x[1], reverse=True)[0:]))

In [ ]:
h1, a1 = nx.hits(G_govt)
print("Hub of Government")
print(max(sorted(h1.items(), key=lambda x: x[1], reverse=True)[0:]))
print("Authority of Government")
print(max(sorted(a1.items(), key=lambda x: x[1], reverse=True)[0:]))

h2, a2 = nx.hits(G_politician)
print("Hub of Politician")
print(max(sorted(h2.items(), key=lambda x: x[1], reverse=True)[0:]))
print("Authority of Politician")
print(max(sorted(a2.items(), key=lambda x: x[1], reverse=True)[0:]))

h3, a3 = nx.hits(G_tv)
print("Hub of TV")
print(max(sorted(h3.items(), key=lambda x: x[1], reverse=True)[0:]))
print("Authority of TV")
print(max(sorted(a3.items(), key=lambda x: x[1], reverse=True)[0:]))

h4, a4 = nx.hits(G_company)
print("Hub of Company")
print(max(sorted(h4.items(), key=lambda x: x[1], reverse=True)[0:]))
print("Authority of Company")
print(max(sorted(a4.items(), key=lambda x: x[1], reverse=True)[0:]))